# Test Local Kinetica Instance

This notebook tests the connectivity and basic operations with the local Kinetica instance.

## Prerequisites
1. Make sure the local Kinetica instance is running:
   ```bash
   cd llm-slicing-5g-lab
   ./run_kinetica_headless.sh
   ```

2. Verify the instance is accessible at http://localhost:8080/gadmin


## 1. Import Required Libraries


In [ ]:
import os
import sys
import time
from datetime import datetime
import pandas as pd
import gpudb
from gpudb import GPUdb
from gpudb import GPUdbColumnProperty as cp
from gpudb import GPUdbRecordColumn as rc
from IPython.display import display, HTML


## 2. Configure Local Kinetica Connection


In [ ]:
# Configure for local Kinetica instance
os.environ["KINETICA_HOST"] = "localhost:9191"
os.environ["KINETICA_USERNAME"] = "admin"
os.environ["KINETICA_PASSWORD"] = "Admin123!"
os.environ["KINETICA_SCHEMA"] = "nvidia_gtc_dli_2025"

print("🔧 Kinetica Configuration:")
print(f"   Host: {os.environ['KINETICA_HOST']}")
print(f"   Username: {os.environ['KINETICA_USERNAME']}")
print(f"   Schema: {os.environ['KINETICA_SCHEMA']}")


## 3. Test Connection


In [ ]:
try:
    # Configure connection options
    kdbc_options = GPUdb.Options()
    kdbc_options.username = os.environ.get("KINETICA_USERNAME")
    kdbc_options.password = os.environ.get("KINETICA_PASSWORD")
    kdbc_options.disable_auto_discovery = True
    
    # Create connection
    kdbc = GPUdb(
        host=os.environ.get("KINETICA_HOST"),
        options=kdbc_options
    )
    
    print("✅ Successfully connected to local Kinetica instance!")
    print(f"   Connection established at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except Exception as e:
    print(f"❌ Failed to connect to Kinetica: {e}")
    print("\n🔍 Troubleshooting:")
    print("1. Make sure the local Kinetica instance is running:")
    print("   cd llm-slicing-5g-lab && ./run_kinetica_headless.sh")
    print("2. Check if the instance is accessible at http://localhost:8080/gadmin")
    print("3. Verify the instance is running on port 9191")
    sys.exit(1)


## 4. Test Basic Operations


In [ ]:
# Test basic operations
try:
    # Get server version
    version_info = kdbc.show_system_properties()
    print("📊 Kinetica Server Information:")
    for prop in version_info:
        if 'version' in prop['property_name'].lower():
            print(f"   {prop['property_name']}: {prop['property_value']}")
    
    # List existing schemas
    schemas = kdbc.show_schemas()
    print(f"\n📁 Available Schemas: {len(schemas)} found")
    for schema in schemas:
        print(f"   - {schema['schema_name']}")
    
    # Check if our target schema exists
    target_schema = os.environ.get("KINETICA_SCHEMA")
    schema_exists = any(s['schema_name'] == target_schema for s in schemas)
    
    if schema_exists:
        print(f"\n✅ Target schema '{target_schema}' exists")
    else:
        print(f"\n⚠️  Target schema '{target_schema}' does not exist")
        print(f"   Creating schema '{target_schema}'...")
        kdbc.create_schema(target_schema)
        print(f"   ✅ Schema '{target_schema}' created successfully")
    
except Exception as e:
    print(f"❌ Error during basic operations: {e}")
    sys.exit(1)


## 5. Create Test Table


In [ ]:
# Create a test table
test_table_name = f"{os.environ.get('KINETICA_SCHEMA')}.test_connectivity_table"

try:
    # Check if table already exists
    if kdbc.has_table(table_name=test_table_name).table_exists:
        print(f"🗑️  Table '{test_table_name}' already exists. Dropping it first...")
        kdbc.drop_table(table_name=test_table_name)
        print(f"   ✅ Table dropped successfully")
    
    # Define table schema
    schema = [
        ["id", rc._ColumnType.STRING, cp.UUID, cp.PRIMARY_KEY, cp.INIT_WITH_UUID],
        ["name", rc._ColumnType.STRING, cp.CHAR8, cp.DICT],
        ["value", rc._ColumnType.FLOAT],
        ["timestamp", rc._ColumnType.STRING, cp.DATETIME, cp.INIT_WITH_NOW],
        ["is_active", rc._ColumnType.INT, cp.INT8]
    ]
    
    # Create table
    print(f"🔨 Creating test table '{test_table_name}'...")
    kdbc_table = gpudb.GPUdbTable(
        _type=schema,
        name=test_table_name,
        db=kdbc
    )
    
    print(f"✅ Test table '{test_table_name}' created successfully!")
    
    # Display table info
    table_info = kdbc.show_table(table_name=test_table_name)
    print(f"\n📋 Table Information:")
    print(f"   Columns: {len(table_info['column_names'])}")
    for i, col_name in enumerate(table_info['column_names']):
        col_type = table_info['column_types'][i]
        print(f"   - {col_name}: {col_type}")
    
except Exception as e:
    print(f"❌ Error creating test table: {e}")
    sys.exit(1)


## 6. Insert Test Data


In [ ]:
try:
    # Prepare test data
    test_data = [
        {"name": "test_record_1", "value": 123.45, "is_active": 1},
        {"name": "test_record_2", "value": 678.90, "is_active": 1},
        {"name": "test_record_3", "value": 234.56, "is_active": 0},
        {"name": "test_record_4", "value": 789.01, "is_active": 1},
        {"name": "test_record_5", "value": 345.67, "is_active": 0}
    ]
    
    print(f"📝 Inserting {len(test_data)} test records...")
    
    # Insert data
    kdbc_table.insert_records(test_data)
    kdbc_table.flush_data_to_server()
    
    print(f"✅ Successfully inserted {len(test_data)} records!")
    
    # Verify insertion
    result = kdbc.get_records_by_column(table_name=test_table_name, column_names=["name", "value", "is_active"], limit=10)
    print(f"\n📊 Sample data from table:")
    
    # Convert to DataFrame for better display
    df = pd.DataFrame({
        'name': result['name'],
        'value': result['value'],
        'is_active': result['is_active']
    })
    
    display(df)
    
except Exception as e:
    print(f"❌ Error inserting test data: {e}")
    sys.exit(1)


## 7. Test Query Operations


In [ ]:
try:
    print("🔍 Testing query operations...")
    
    # Test 1: Count total records
    count_result = kdbc.execute_sql(f"SELECT COUNT(*) as total_records FROM {test_table_name}")
    total_count = count_result['total_records'][0]
    print(f"   📊 Total records: {total_count}")
    
    # Test 2: Count active records
    active_result = kdbc.execute_sql(f"SELECT COUNT(*) as active_records FROM {test_table_name} WHERE is_active = 1")
    active_count = active_result['active_records'][0]
    print(f"   ✅ Active records: {active_count}")
    
    # Test 3: Average value
    avg_result = kdbc.execute_sql(f"SELECT AVG(value) as avg_value FROM {test_table_name}")
    avg_value = avg_result['avg_value'][0]
    print(f"   📈 Average value: {avg_value:.2f}")
    
    # Test 4: Max value
    max_result = kdbc.execute_sql(f"SELECT MAX(value) as max_value FROM {test_table_name}")
    max_value = max_result['max_value'][0]
    print(f"   🔝 Max value: {max_value}")
    
    print("\n✅ All query operations completed successfully!")
    
except Exception as e:
    print(f"❌ Error during query operations: {e}")
    sys.exit(1)


## 8. Test Table Deletion


In [ ]:
try:
    print(f"🗑️  Testing table deletion...")
    
    # Verify table exists before deletion
    if kdbc.has_table(table_name=test_table_name).table_exists:
        print(f"   ✅ Table '{test_table_name}' exists")
        
        # Drop the table
        kdbc.drop_table(table_name=test_table_name)
        print(f"   🗑️  Table '{test_table_name}' dropped successfully")
        
        # Verify table no longer exists
        if not kdbc.has_table(table_name=test_table_name).table_exists:
            print(f"   ✅ Confirmed: Table '{test_table_name}' no longer exists")
        else:
            print(f"   ⚠️  Warning: Table '{test_table_name}' still exists after deletion")
    else:
        print(f"   ⚠️  Table '{test_table_name}' does not exist")
    
    print("\n✅ Table deletion test completed!")
    
except Exception as e:
    print(f"❌ Error during table deletion: {e}")
    sys.exit(1)


## 9. Test Summary


In [ ]:
print("🎉 Kinetica Local Instance Test Summary")
print("=" * 50)
print(f"✅ Connection: SUCCESSFUL")
print(f"✅ Schema Operations: SUCCESSFUL")
print(f"✅ Table Creation: SUCCESSFUL")
print(f"✅ Data Insertion: SUCCESSFUL")
print(f"✅ Query Operations: SUCCESSFUL")
print(f"✅ Table Deletion: SUCCESSFUL")
print("\n🚀 Your local Kinetica instance is working perfectly!")
print("\n📋 Next Steps:")
print("1. You can now run the main DLI_Lab_Setup.ipynb notebook")
print("2. Access the admin console at http://localhost:8080/gadmin")
print("3. Use the agentic-llm tools with confidence")
print(f"\n⏰ Test completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## 10. Troubleshooting

If you encounter any issues:

### Connection Issues
1. **Check if Kinetica is running**:
   ```bash
   cd llm-slicing-5g-lab
   ./run_kinetica_headless.sh
   ```

2. **Verify ports are accessible**:
   - Admin Console: http://localhost:8080/gadmin
   - Database REST: http://localhost:9191

3. **Check Docker containers**:
   ```bash
   docker ps | grep kinetica
   ```

### Authentication Issues
- Username: `admin`
- Password: `Admin123!`

### Schema Issues
- Default schema: `nvidia_gtc_dli_2025`
- The test will create the schema if it doesn't exist

### Performance Issues
- Ensure sufficient system resources
- Check Docker memory allocation
- Monitor system performance during operations
